## Spoznavanje z okoljem

In [ ]:
import pandas as pd
import cobra

from cobra.io import load_model

import numpy as np
import pickle

import os.path 

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import HTML

## *E. coli*

Preberimo model metabolizma *E. coli*. Model smo prenesli s strani http://bigg.ucsd.edu/models/e_coli_core.

In [ ]:
model = cobra.io.read_sbml_model(os.path.join('models','e_coli_core.xml'))

In [ ]:
model

Nastavimo funkcijo biomase:

In [ ]:
r_biomass = model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
r_biomass

In [ ]:
model.objective = r_biomass

In [ ]:
model.objective.expression

In [ ]:
sol = model.optimize()

In [ ]:
model.summary()

Rast celice torej znaša 0.874 $h^{-1}$.

Rezultat optimizacije lahko primerjamo z orodjem [Escher-FBA](https://sbrg.github.io/escher-fba/#/app):

In [ ]:
sol = model.optimize()
sol['EX_glc__D_e']

### Medij gojenja

Poskusimo spremeniti lastnosti medija. Kot vidimo zgoraj, celica črpa veliko glukoze (pretok je 10 $mmol gDW^{-1} hr^{-1}$ (milimoli na gram suhe celične mase na uro - privzete enote v okolju COBRA)). Kakšen je maksimalen dovod glukoze? Ali celica črpa vse, kar ima na voljo?

In [ ]:
r_ex_glc = model.reactions.get_by_id('EX_glc__D_e')
r_ex_glc

Pretok je negativen, ampak reakcija je obrnjena! V orodju COBRA so vse *exchange* (*uptake* in *secretion*) reakcije take oblike. Kot vidimo, celica črpa vso glukozo, ki je na voljo. Poskusimo to zmanjšati:

In [ ]:
r_ex_glc.bounds

In [ ]:
r_ex_glc.bounds = (0, 1000.0)
model.optimize()

Ups. Tole je bilo preveč. 

In [ ]:
r_ex_glc.bounds = (-1.0, 1000.0)

In [ ]:
model.optimize()

Poskusimo analizo ponoviti v anaerobnih pogojih. Nastavili bomo *uptake* kisika:

In [ ]:
r_ex_o2 = model.reactions.get_by_id('EX_o2_e')
r_ex_o2

In [ ]:
r_ex_glc.bounds = (-10.0, 1000.0)
r_ex_o2.bounds = (0, 1000)

In [ ]:
model.optimize()

Produkcija biomase se je zmanjšala, prav tako pa se je preusmerila pot generiranja energije. Celica tvori in izloča acetat (`ac`), format (`for`) in etanol (`etoh`). Prej je izločala vodo in ogljikov dioksid.

In [ ]:
model.summary()

In [ ]:
s.objective_value

Lahko naredimo analizo produkcije biomase glede na razpoložljivost kisika in glukoze:

In [ ]:
o2_max = 30
glc_max = 30

# ustvarimo urejen seznam razpon vrednosti kisika in glukoze, za katere želimo narediti analizo
o2_range = np.arange(0, o2_max+1)
glc_range = np.arange(0, glc_max+1)

# ustvarimo matriko z dvema vrsticama - 1. za kisik, 2. za glukozo
# v vsaki vrstici je toliko ničel kot je število vrednosti, za katere bomo naredili analizo
# ničle se bodo tekom računanja spreminjale v ustrezna števila glede na rezultate analize
B = np.zeros((len(o2_range),len(glc_range)))

# za vsak par vrednosti o2 in glc se bo v matriko vpisal rezultat optimizacije
for i,o2 in enumerate(o2_range):
    for j,glc in enumerate(glc_range):
        r_ex_o2.lower_bound = -o2 # nastavitev spodnje meje reakcij privzema kisika
        r_ex_glc.lower_bound = -glc # nastavitev spodnje meje reakcij prvizema glukoze
        s = model.optimize()
        B[i,j] = s.objective_value # vpis rezultata v matriko

In [ ]:
ax = sns.heatmap(B)
ax.invert_yaxis()
plt.xlabel("glc")
plt.ylabel("o2")
plt.show()

## FVA
FVA (Flux Variability Analysis) omogoča analizo razpona pretokov skozi rekacije pri (skoraj) optimalni produkciji biomase. 

In [ ]:
from cobra.flux_analysis import flux_variability_analysis

In [ ]:
fva = flux_variability_analysis(model, fraction_of_optimum=0.9, loopless=True)

In [ ]:
fva

In [ ]:
model.optimize()
model.summary(fva=0.90)

### Geni v modelu

In [ ]:
for r in model.reactions:
    print(r.gpr)

In [ ]:
model.genes.get_by_id('b4015')

## Recon3D

Preberimo še model Recon3D. Predtem smo ga prenesli s strani [https://github.com/SBRG/Recon3D](https://github.com/SBRG/Recon3D). Opis modela je na voljo v [članku](http://dx.doi.org/10.1038/nbt.4072).

In [ ]:
model = cobra.io.read_sbml_model(os.path.join('models','Recon3D.xml'))

In [ ]:
model

Poglejmo nekaj atributov modela:

In [ ]:
model.genes[:3]

In [ ]:
model.metabolites[:3]

In [ ]:
model.reactions[:3]

Povzetek modela lahko dobimo z metodo [summary](https://cobrapy.readthedocs.io/en/latest/autoapi/cobra/summary/model_summary/index.html):

In [ ]:
model.summary()

In [ ]:
for r in model.reactions[:100]:
    gpr = r.gene_reaction_rule
    if gpr:
        print(gpr)

In [ ]:
g = model.genes.get_by_id('55902_AT1')

Izpišimo gene, ki nastopajo v vsaj 5 reakcijah:

In [ ]:
d = {}

for g in model.genes:
    d[g.name] = len(g.reactions)

In [ ]:
sorted(list(d.items()), key=lambda x: x[1], reverse = True)